In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

path = '/usr/local/bin/chromedriver'
website = 'https://indiarailinfo.com/trains/longestroutes'
service = Service(executable_path=path)
options = Options()
driver = webdriver.Chrome(service=service, options=options)
driver.get(website)

# Keep clicking "Next" until no more pages
while True:
    try:
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@class="nextbtn"]'))
        )
        next_button.click()
        time.sleep(1)  # wait for data to load
    except:
        break

Found 7338 entries


In [11]:

from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Get updated HTML from Selenium
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Step 2: Locate the container/table
table = soup.find('div', class_="srhres newbg inline alt")

# Step 3: Extract column titles (assuming they are in the 3rd div)
header_div = table.find_all('div')[2]
title = [t.text.strip() for t in header_div.find_all('div')]
print("Column titles:", title)

# Step 4: Create DataFrame with those column names
df = pd.DataFrame(columns=title)

# Step 5: Extract rows of data (from the 5th div onward)
rows = table.find_all('div')[4:]
for row in rows:
    row_data = row.find_all('div')
    individual_row_data = [data.text.strip() for data in row_data]
    
    # Only add rows that have actual data
    if individual_row_data and any(item != '' for item in individual_row_data):
        if len(individual_row_data) == len(title):  # Ensure column count matches
            df.loc[len(df)] = individual_row_data
        else:
            print("Skipping row due to column mismatch:", individual_row_data)

# Step 6: Show the final DataFrame
df


Column titles: ['No.', 'Name', 'Type', 'Zone', 'Date From', 'Date To', 'From', 'Dep', 'To', 'Arr', 'Duration', 'Halts', 'Dep Days', 'Classes', 'Distance', 'Speed', 'Return']
Skipping row due to column mismatch: ['Refresh Live Status (not updated)Slowed down: 5m. TT Updated: Jan 01 (15:48)Compare with Old TT20829/Durg - Visakhapatnam Vande Bharat Express\xa0\xa0\xa0\xa0\xa0दुर्ग - विशाखपट्टणम वंदे भारत एक्सप्रेसThis is the Shortest Route between Durg Jn and Visakhapatnam Jn.Travel Time: 8h 5m\xa0\xa0\xa0\xa09 halts\xa0\xa0\xa0\xa0Distance: 567 km\xa0\xa0\xa0\xa0Avg Speed: 70 km/hrMax Permissible Speed: 130 km/hrType: Vande BharatRake Zone: SECR/South East CentralDeparts @ 05:45Platform# --SMTWTFSDURG/Durg Jn\xa0Arrives @ 13:50 Platform# --SMTWTFSVSKP/Visakhapatnam Jn\xa0Other Trains in the NewsWL Quota: n/aCharting @ 21:00DateAC ChairExecutive CCFreshnessRefresh\xa0\xa0n/a\xa0n/a\xa0n/a\xa0n/a\xa0n/a\xa0n/a\xa0n/a\xa0n/a\xa0\xa0Tot Seats\xa0\xa0UpdateFri-Jun 06\xa0\xa0-RefreshSat-Jun 07

,No.,Name,Type,Zone,Date From,Date To,From,Dep,To,Arr,Duration,Halts,Dep Days,Classes,Distance,Speed,Return
0,22504,Dibrugarh - Kanniyakumari Vivek SF Express (PT),SF,NFR,,,DBRG,19:35,CAPE,21:55,74h 20m,57,SMTWTFS,SL 3A 2A,4155 km,56 km/hr,22503
1,22503,Kanniyakumari - Dibrugarh Vivek SF Express (PT),SF,NFR,,,CAPE,17:25,DBRG,21:00,75h 35m,57,SMTWTFS,SL 3A 2A,4154 km,55 km/hr,22504
2,12507,Aronai Express (PT),SF,NFR,,,TVC,16:55,SCL,16:15,71h 20m,55,SMTWTFS,SL 3A 2A 3E,3916 km,55 km/hr,12508
3,12508,Aronai Express (PT),SF,NFR,,,SCL,19:30,TVC,20:45,73h 15m,54,SMTWTFS,SL 3A 2A 3E,3916 km,53 km/hr,12507
4,16318,Himsagar Express (PT),Exp,SR,,,SVDK,22:30,CAPE,23:05,72h 35m,67,SMTWTFS,SL 3A 2A 3E,3789 km,52 km/hr,16317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3663,06527,Bangarapet - SMVT Bengaluru MEMU Express (UnRe...,Exp,SWR,,,BWT,20:40,SMVB,22:15,1h 35m,5,SMTWTFS,,61 km,38 km/hr,06528
3664,56136,Mettupalayam - Udagamandalam MG Passenger,Toy,SR,,,MTP,07:10,UAM,11:55,4h 45m,4,SMTWTFS,2S FC,46 km,10 km/hr,56137
3665,56137,Udagamandalam - Mettupalayam MG Passenger,Toy,SR,,,UAM,14:00,MTP,17:30,3h 30m,4,SMTWTFS,2S FC,46 km,13 km/hr,56136
3666,17264,Narsapur - Bhimavaram Junction MEMU Express (U...,Exp,SCR,,,NS,22:45,BVRM,23:30,0h 45m,2,SMTWTFS,,30 km,39 km/hr,17263


In [13]:
# converting to the csv 
df.to_csv(r'/Users/apple/Downloads/train_route_2.csv',index=False)